# _Machine learning_ y el Titanic
En el proyecto del tercer día haremos algunas predicciones sobre una tragedia que todos conocemos: el hundimiento del Titanic. Eso significa más _machine learning_, pero a diferencia de los dos primeros días, no haremos más _clustering_. Haremos uso de __regresión logística__ y analizaremos las variables más importantes para determinar la supervivencia. Sí, vamos a desarrollar un modelo que prediga qué pasajeros sobrevivieron. 

El _DataFrame_ con los datos está contenido en un archivo de valores separados por coma llamado `train.csv`.

In [1]:
# Pandas o 'Panel data', nuestro mejor amigo.
import pandas as pd

# Llamaremos a las variables de los DataFrames así como el archivo de coma separada.
train = pd.read_csv('train.csv')

# Familiarización con la tabla.
print('Éste es el DataFrame con los datos con los que entrenaremos al modelo:')
display(train.head())

Éste es el DataFrame con los datos con los que entrenaremos al modelo:


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Limpieza y organización de datos
Para empezar con las modificaciones y poder entrenar correctamente a nuestro modelo, vamos a pasar los nombres de las columnas a la convención _snake_case_. Después explicaré qué significan algunas de ellas.

In [2]:
# Pasamos los nombres de las columnas a minúsculas para cada DataFrame.
train.columns = train.columns.str.lower()

# Cambiemos estas tres columnas para mayor legibilidad.
train = train.rename(columns={
    'passengerid': 'passenger_id',
    'pclass': 'p_class',
    'sibsp': 'sib_sp',
    'parch': 'par_ch'
})

# Comprobación
print(train.columns)

Index(['passenger_id', 'survived', 'p_class', 'name', 'sex', 'age', 'sib_sp',
       'par_ch', 'ticket', 'fare', 'cabin', 'embarked'],
      dtype='object')


Esto empieza a tener más orden. Vamos a conocer las columnas y sus significados. Me limitaré a explicar columnas que podrían causar confusión. Es evidente el significado de columnas como `'sex'` o `'name'`.

- `'passenger_id'` es el identificador de cada pasajero.
- `'p_class'` es la clase del _ticket_; primera, segunda o tercera y se representa con los números 1, 2 y 3.
- `'sib_sp'` es el número de hermanos y esposos/as.
- `'par_ch'` es el número de padres e hijos.
- `'ticket'`es el número de _ticket_.
- `'fare'` fue la tarifa pagada por subir.
- `'cabin'` es el código del camarote, que puede incluir nulos.
- `'embarked'` es el puerto de embarque. Hay tres opciones: `C` significa Cherbourg; `Q` significa Queenstown y `S` Southampton.

Vamos a inspeccionar esos nulos y si es que hay registros duplicados.

In [3]:
# Hay que darle algo de formato a estas impresiones, por qué no.
print()
print()
print('¿Qué tipos de datos hay en las columnas?')
print('_________________________________________')
print(train.info())

print()
print()
print('¿Cuántos nulos hay por columna?')
print('________________________________')
print(train.isnull().sum())

print()
print()
print('¿Cuántos datos duplicados hay?')
print('___________________________')
print(train.duplicated().sum())



¿Qué tipos de datos hay en las columnas?
_________________________________________
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   passenger_id  891 non-null    int64  
 1   survived      891 non-null    int64  
 2   p_class       891 non-null    int64  
 3   name          891 non-null    object 
 4   sex           891 non-null    object 
 5   age           714 non-null    float64
 6   sib_sp        891 non-null    int64  
 7   par_ch        891 non-null    int64  
 8   ticket        891 non-null    object 
 9   fare          891 non-null    float64
 10  cabin         204 non-null    object 
 11  embarked      889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


¿Cuántos nulos hay por columna?
________________________________
passenger_id      0
survived          0
p_class           0
name         

Los tipos de datos de nuestra tabla tienen sentido. Hay datos nulos en las columnas ``age``, `'cabin'` y `'embarked'`. Para las edades vamos a sustituir los nulos por la mediana (ya que la media podría quedarse corta en el sentido de que la distribución de edades no es simétrica). La columna del camarote deberá ser eliminada, pues tiene 687 nulos y afectaría a la calidad de nuestro modelo. Y para la columna de embarcación solamente hay dos nulos, así que los vamos a sustituir por la moda (el valor más frecuente). No hay duplicados.

In [4]:
# Sustitución de edades nulas por la mediana.
train['age'] = train['age'].fillna(train['age'].median())

# Eliminación de la columna 'cabin'.
train = train.drop('cabin', axis=1)

# Sustitución de embarque nulo con la moda.
train['embarked'] = train['embarked'].fillna(train['embarked'].mode()[0])

# Comprobación de duplicados
print(train.duplicated().sum())


0


Ya casi terminamos de limpiar y organizar los datos. Ahora vamos a pasar algunos de los datos a valores numéricos. Hablo de los datos de las columnas `'sex'` y `'embarked'`.

In [5]:
# Con la función `map()` podemos tranformar los valores de una columna de acuerdo a un diccionario.
train['sex'] = train['sex'].map({'male': 0, 'female': 1})
train['embarked'] = train['embarked'].map({'S': 0, 'C': 1, 'Q': 2})

# Comprobación.
print(train.head())

   passenger_id  survived  p_class  \
0             1         0        3   
1             2         1        1   
2             3         1        3   
3             4         1        1   
4             5         0        3   

                                                name  sex   age  sib_sp  \
0                            Braund, Mr. Owen Harris    0  22.0       1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...    1  38.0       1   
2                             Heikkinen, Miss. Laina    1  26.0       0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)    1  35.0       1   
4                           Allen, Mr. William Henry    0  35.0       0   

   par_ch            ticket     fare  embarked  
0       0         A/5 21171   7.2500         0  
1       0          PC 17599  71.2833         1  
2       0  STON/O2. 3101282   7.9250         0  
3       0            113803  53.1000         0  
4       0            373450   8.0500         0  


Ya nos hay datos nulos ni duplicados. Ahora tenemos que pensar, ¿qué columnas podría darle información válida a mi modelo predictivo? Vamos a dejar de lado los nombres de las personas, su identificador y su número de ticket. No nos aportan información relevante. Así que filtraremos las características relevantes. Estamos por empezar el entrenamiento de nuestro modelo.

## Entrenamiento del modelo de regresión logística
Ahora viene lo más interesante. Determinemos las varibles que utilizaremos y empecemos a entrenar nuestro modelo.

In [6]:
# Hagamos una lista con las columnas que usaremos como características para nuestro modelo.
features = ['p_class', 'sex', 'age', 'sib_sp', 'par_ch', 'fare', 'embarked']

# Hagamos las variables para el modelo. Características y variable objetivo.
X = train[features]
y = train['survived']

Ahora dividiremos nuestro _DataFrame_ en dos partes: entrenamiento y validación.

In [7]:
# Usaremos la librería más popular de machine learning en Python, `sklearn`.
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


Nuestro modelo, como dije al inicio del proyecto, será el de de regresión logística. Este modelo estadístico se utiliza para predecir la probabilidad de que ocurra un evento con dos posibles resultados (sí o no, 0 o 1). Es simple, claro y eficiente.

En este caso añadir un escalador no provoca ningún cambio en la precisión de las predicciones, así que no añadiremos uno.

In [8]:
# Aquí estamos importando nuestro modelo de regresión lógistica.
from sklearn.linear_model import LogisticRegression

# Asignamos una variable al modelo y lo ajustamos.
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)


LogisticRegression(max_iter=1000)

Vamos a hacer que el modelo prediga y vamos a medir su precisión.

In [9]:
from sklearn.metrics import accuracy_score

y_pred = model.predict(X_val)
print(f"Precisión en validación: {accuracy_score(y_val, y_pred) * 100:.2f}%")


Precisión en validación: 79.89%


Así que desarrollamos un modelo con una __precisión de casi el 80%__. Nada mal, ¿no? Pero ahora me surge otra duda. ¿Cuáles de las características que seleccionamos tienen un mayor peso en la predicción de nuestro modelo? Tengo algunas hipótesis: seguramente una menor edad, ser de sexo femenino y ser adinerado pudieron ser factores a favor de la supervivencia en el Titanic. Comprobémoslo y comprendamos los pesos de las variables.

In [10]:
# Extracción de los coeficientes y los nombres de las variables.
feature_weights = pd.DataFrame({
    'feature': features,
    'weight': model.coef_[0]
})

# Ordenemos de mayor a menor peso.
feature_weights['abs_weight'] = feature_weights['weight'].abs()
feature_weights = feature_weights.sort_values('abs_weight', ascending=False)

# Impresión de los resultados.
print(feature_weights[['feature', 'weight']])

    feature    weight
1       sex  2.581610
0   p_class -0.958071
3    sib_sp -0.302852
6  embarked  0.222488
4    par_ch -0.100745
2       age -0.031165
5      fare  0.002863


## Conclusión final

- El __sexo__ (`sex`: 2.58) es el factor con más peso. Ser mujer (1) incrementa mucho la probabilidad de supervivencia comparado con ser hombre (0). Es lo que esperábamos.

- Pertenecer a una __clase más baja__  (`p_class`: -0.95) disminuye la probabilidad de sobrevivir. También lo esperábamos.

- Tener más __hermanos o esposos/as__ viajando juntos (`sib_sp`: -0.30) reduce ligeramente la probabilidad de sobrevivir. Así que mejor viajar solito en barco. Jeje.

- Haber __embarcado en ciertos puertos__ (`embarked`: 0.22) parece aumentar algo la probabilidad de sobrevivir. El efecto es pequeño comparado con sexo o clase social.

- Más __padres e hijos__ a bordo (`par_ch`: -0.10) disminuye ligeramente la probabilidad de sobrevivir, como en la variable de los hermanos y esposo/sa.

- La __edad__ tiene poco peso (`age`: -0.03), pero aumenta levemente la probabilidad de no sobrevivir si es mayor. La verdad esperaba un mayor peso de la edad

- El __precio__ pagado por el boleto (`fare`: 0.002) tiene un impacto muy pequeño en la predicción, apenas afecta la probabilidad de sobrevivir.

Como hemos comprobado, un modelo simple de regresión lineal nos ayuda a hacer predicciones bastante acertadas, además de determinar qué características son las de mayor peso. ¿Interesante, no?